# Critic Pretraining

This notebook serves to demostrate how to pretrain the critic (from some source of generated data).

## TODO
1. Update shared training loop to take in a train step function.
3. Update script_testing to be compatiable.
2. Update names to Actors.
4. LOTS

# Imports/Setup

In [1]:
from Data import critic_dataset
from Data import utility as data_utility
from Model import critics
from Model import utility as model_utility

# Configuration

In [12]:
class Config:
    # device
    device_pref = 'cuda'
    ind_dev = 3
    
    # dataset
    directory = '/data/users/jupyter-dam724/RL/precondition-discovery-contextual-bandit/Buffer'
    cache = False
    seed = 42
    ratio = 0.8
    shuffle = True
    workers = 16
    pin = False
    postprocess = critic_dataset.postprocess
    
    # model
    n = 25
    down = 256 # if this is change, model architecture should be changed as well
    batch_size = 32
    sigmoid = False

# Device

In [13]:
device = data_utility.set_device(Config.device_pref, Config.ind_dev)

Now using GPU.


# Critics Dataset

Instantiate the saved files during hyperparameter tuning to a torch dataset. Caching and dataloading is recommended for full data read in speed.

In [14]:
files = model_utility.get_torch_files(Config.directory)

In [15]:
split_params = {
    'files': files, 
    'train_ratio': Config.ratio, 
    'random_seed': Config.seed
}

train, valid = data_utility.train_valid_split(**split_params)

Total files: 50623
Training files: 40498 (80.0%)
Validation files: 10125 (20.0%)


In [16]:
train_dset_params = {
    'files': train, 
    'cache': Config.cache, 
    'root_dir': Config.directory                     
}
valid_dset_params = {
    'files': valid, 
    'cache': Config.cache, 
    'root_dir': Config.directory                     
}

t_dataset = critic_dataset.CriticDataset(**train_dset_params)
v_dataset = critic_dataset.CriticDataset(**valid_dset_params)

In [17]:
train_dl_params = {
    'dataset': t_dataset, 
    'batch_size': 1, 
    'shuffle': Config.shuffle, 
    'num_workers': Config.workers, 
    'pin_memory': Config.pin
}
valid_dl_params = {
    'dataset': v_dataset, 
    'batch_size': 1, 
    'shuffle': Config.shuffle, 
    'num_workers': Config.workers, 
    'pin_memory': Config.pin
}

train_dl = critic_dataset.get_dataloader(**train_dl_params)
valid_dl = critic_dataset.get_dataloader(**valid_dl_params)

# Model

Create the model for the critic. Two models are accessible: single output critc and multi ouput critic. The single and multi refer to how the output the measures of reward (combined versus seperate). Each variant requires a differnt `step` and `loss`.

In [20]:
model_params = {
    'n': Config.n, 
    'down': Config.down, 
    'batch_size': Config.batch_size, 
    'sigmoid_scale': Config.sigmoid
}

critic = critics.MultiCritic(**model_params).to(device)